# Very naive deep learning on the vector of surrounding bases

---

### Data

Naive feature vectors. The original sequence of validation/test and train data does not overlap! ( but train data points can overlap with train data points, and test-validation can overlap with test-validation data ) This overlapping does not lead to unintentional label leakage!



### Notes


---

Instruct theano to use gpu

In [1]:
import os
os.environ['THEANO_FLAGS']='device=gpu'

import sys
sys.path.append('../my_modules')
from loading_utils import read_my_data

import subprocess
import time

import os,subprocess
workdir='/mnt/Data1/ribli/methylation_code/modelling'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

Using gpu device 0: GeForce GTX 670 (CNMeM is disabled, CuDNN not available)


### Load data

In [2]:
_,train_x,train_y = read_my_data(
    fname='../prepare_data/big_train_feat_vect.csv')
test_id,test_x,test_y = read_my_data(
    fname='../prepare_data/big_test_feat_vect.csv')

#make it image like
train_x,test_x=[x.reshape((-1,1,1000,1)) for x in (train_x,test_x)]

Loading data... 
Loading data... 


### Build Convnet

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D,MaxPooling2D

input_dim=train_x.shape[2]
activation='relu'
loss='binary_crossentropy'
optimizer='adadelta'
init='uniform'
pool_size=(8,1)
window_size=5
dense_n=64

model = Sequential()

#Convolution layer 1
model.add(Convolution2D(20,window_size,1, border_mode='valid',input_shape=(1,input_dim,1)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Convolution layer 2
model.add(Convolution2D(50,window_size,1, border_mode='valid'))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Dense layer
model.add(Flatten())
model.add(Dense(dense_n,activation=activation))
#model.add(Dropout(0.5))

#final layer
model.add(Dense(1, activation='sigmoid'))

#compile model
model.compile(loss=loss,optimizer=optimizer,class_mode='binary')

Using Theano backend.
/mnt/Data1/ribli/tools/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


### Train and test model

In [4]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

def fit_keras_model(model,train_x,train_y,test_x,test_y):
    start=time.time()
    
    #callbacks
    best_model=ModelCheckpoint('best_model',save_best_only=True,verbose=1)
    early_stop=EarlyStopping(patience=7,verbose=1)
    
    #train it
    callb_hist=model.fit(train_x,train_y,nb_epoch = 100,
                         show_accuracy=True,verbose=1,
                        validation_split=0.05,
                        callbacks=[best_model,early_stop])
    #predict
    model.load_weights('best_model')
    train_pred=model.predict_classes(train_x).ravel()
    test_pred=model.predict_classes(test_x).ravel()

    #check errors
    print 'train score:',list((train_pred==train_y)).count(True)/float(len(train_y))
    print 'test score:',list((test_pred==test_y)).count(True)/float(len(test_y))

    print 'It took:',time.time()-start    
    return train_pred,test_pred

In [5]:
N_train,N_test=train_x.shape[0],test_x.shape[0]
train_pred,test_pred=fit_keras_model(
    model,train_x[:N_train],train_y[:N_train],test_x[:N_test],test_y[:N_test])

Train on 213117 samples, validate on 11217 samples
Epoch 1/100
213117/213117 [==============================] - 83s - loss: 0.4855 - acc: 0.7699 - val_loss: 0.3860 - val_acc: 0.8351
Epoch 00000: val_loss improved from inf to 0.38602, saving model to best_model
Epoch 2/100
213117/213117 [==============================] - 83s - loss: 0.3898 - acc: 0.8338 - val_loss: 0.3570 - val_acc: 0.8538
Epoch 00001: val_loss improved from 0.38602 to 0.35696, saving model to best_model
Epoch 3/100
213117/213117 [==============================] - 83s - loss: 0.3573 - acc: 0.8537 - val_loss: 0.5334 - val_acc: 0.7621
Epoch 00002: val_loss did not improve
Epoch 4/100
213117/213117 [==============================] - 83s - loss: 0.3402 - acc: 0.8620 - val_loss: 0.3401 - val_acc: 0.8624
Epoch 00003: val_loss improved from 0.35696 to 0.34008, saving model to best_model
Epoch 5/100
213117/213117 [==============================] - 83s - loss: 0.3313 - acc: 0.8674 - val_loss: 0.3038 - val_acc: 0.8829
Epoch 00004

### Save test predictions

In [10]:
#load best model
model.load_weights('best_model')
test_pred=model.predict_classes(test_x).ravel()

import pandas as pd
import numpy as np
result=pd.DataFrame({'id':test_id,'label':test_y,'prediction':test_pred})
result['error']=np.abs(result['label']-result['prediction'])
result.head()

10000/10000 [==============================] - 1s     


,id,label,prediction,error
0,cg02979850,1,1,0
1,cg02979170,0,1,1
2,cg24913115,0,0,0
3,cg02118630,1,1,0
4,cg00393407,1,1,0


In [11]:
result.to_csv('cnn_test_preds.csv',sep='\t',index=False,Header=True)